# Optimisation via Simulation Tutorial

By Christine S.M Currie and Tom Monks

In [1]:
import numpy as np

## Models

To do.

* Need to rename some of the functions so that it fits
* Need to adapt Christine's Law inventory model

In [2]:
from ovs.toy_models import (custom_guass_bandit_problem,
                            BanditCasino, 
                            guassian_bandit_sequence)

## Procedure **KN**

To run Kim and Nelson's R&S procedure KN, create an instance of `ovs.indifference_zone.KN`

An object of type KN takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **delta** - float, the indifference zone
* **alpha** - float, $PCS = 1-\alpha$ (default=0.05)
* **n_0** - int, $n_0$ the number of initial replications (default=2)

In [3]:
from ovs.indifference_zone import KN

In [4]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.1, 
        alpha=0.05, 
        n_0=5)

In [5]:
best_design = kn.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(kn._actions))
print('total reps\t{0}'.format(kn._actions.sum()))
print('means\t\t{0}'.format(kn._means))

best design	[9]
allocations	[ 15 164  30  10  11 239  42  43  44 240]
total reps	838
means		[0.86741622 2.05419091 3.18459728 4.03429256 4.86059118 6.06445448
 6.89535764 8.14361863 9.33728733 9.94901551]


## Optimal Computing Budget Allocation (OCBA)

An object of type OCBA takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [6]:
from ovs.fixed_budget import OCBA

In [7]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=300, 
            delta=10, 
            n_0=5, 
            min=True)

call the `solve()` method to run the optimisation

In [8]:
results = ocba.solve()
print('best design:\t{}'.format(results))
print('allocations:\t{}'.format(ocba._allocations))
print('total reps:\t{}'.format(ocba._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocba._means))
print('vars:\t\t{0}'.format(ocba._vars))

best design:	0
allocations:	[255   5   5   5   5   5   5   5   5   5]
total reps:	300
means:		[ 0.95  2.88  2.94  3.56  5.17  5.83  6.58  8.51  8.86 10.13]
vars:		[0.94 0.36 0.51 0.12 1.16 1.73 1.24 0.99 0.2  0.4 ]


/home/tom/Dropbox/Python/ovs-tutorial/ovs/fixed_budget.py:202: RuntimeWarning: divide by zero encountered in true_divide
  (budget_to_allocate / ratio_s * self._ratios[more_runs])


# Evaluation of the methods

In [9]:
from ovs.evaluation import Experiment

In [10]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.1, 
        alpha=0.2, 
        n_0=5)

exp = Experiment(env=environment, procedure=kn, best_index=9, replications=10)

In [11]:
results = exp.execute()

In [12]:
results.p_correct_selections

0.9

In [13]:
results.selections

array([9, 9, 9, 9, 8, 9, 9, 9, 9, 9], dtype=int32)

In [14]:
results.correct_selections

9

In [15]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=200, 
            delta=10, 
            n_0=5, 
            min=False)

exp = Experiment(env=environment, procedure=ocba, best_index=9, replications=50)

In [16]:
results = exp.execute()

/home/tom/Dropbox/Python/ovs-tutorial/ovs/fixed_budget.py:202: RuntimeWarning: divide by zero encountered in true_divide
  (budget_to_allocate / ratio_s * self._ratios[more_runs])


In [17]:
results.p_correct_selections

1.0

In [18]:
results.selections

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9], dtype=int32)

In [19]:
results.correct_selections

50

## Try different budgets

In [20]:
from ovs.evaluation import GridExperiment

In [34]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

param_grid = {'model':[environment],
              'budget':[100, 200, 300, 400, 500], 
              'n_designs':[len(designs)],
              'delta':[1, 5,10],
              'n_0':[5],
              'min':[False]
              }


exp = GridExperiment(agent=ocba, 
                     environment=environment, 
                     param_grid=param_grid,
                     best_index=9,
                     replications=100)

In [35]:
results = exp.fit()

In [36]:
results

,n_designs,n_0,model,min,delta,budget,correct_selections,p_correct_selections
0,10,5,<ovs.toy_models.BanditCasino object at 0x7f2f1...,False,1,100,500,1
1,10,5,<ovs.toy_models.BanditCasino object at 0x7f2f1...,False,5,100,500,1
2,10,5,<ovs.toy_models.BanditCasino object at 0x7f2f1...,False,10,100,500,1
3,10,5,<ovs.toy_models.BanditCasino object at 0x7f2f1...,False,1,200,500,1
4,10,5,<ovs.toy_models.BanditCasino object at 0x7f2f1...,False,5,200,500,1
5,10,5,<ovs.toy_models.BanditCasino object at 0x7f2f1...,False,10,200,500,1
6,10,5,<ovs.toy_models.BanditCasino object at 0x7f2f1...,False,1,300,500,1
7,10,5,<ovs.toy_models.BanditCasino object at 0x7f2f1...,False,5,300,500,1
8,10,5,<ovs.toy_models.BanditCasino object at 0x7f2f1...,False,10,300,500,1
9,10,5,<ovs.toy_models.BanditCasino object at 0x7f2f1...,False,1,400,500,1
